# PREDICTION

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle as pkl
import pandas as pd
import numpy as np

#### Loading Trained Model, Scaler and Encoders pkl file

In [4]:
# Gender ENcoder
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pkl.load(file)

# Geography Encoder
with open('ohe_geo.pkl', 'rb') as file:
    ohe_geo = pkl.load(file)

# Scaler 
with open('scaler.pkl', 'rb') as file:
    scaler = pkl.load(file)

# Trained Model
model = load_model('model.h5')

In [5]:
# Example Input Data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000    
}

In [6]:
# Encoding Geography
geo_encoded = ohe_geo.transform([[input_data['Geography']]])

geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\shash\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
# Convert input_data into DataFrame
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
# Encoding gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
# Combine OHE encoded columns with input data
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
# Scaling the input data
input_scaled = scaler.transform(input_df)

In [11]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

## Prdecition

In [27]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


array([[0.0298396]], dtype=float32)

In [31]:
prediction_probability = prediction[0][0]
prediction_probability

0.0298396

In [35]:
if prediction_probability > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
